In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline
from astropy.stats import sigma_clipped_stats, mad_std
from photutils import DAOStarFinder, CircularAperture, aperture_photometry, CircularAnnulus
from astropy.io import fits
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
import astropy.io.fits as pf
plt.rcParams['figure.figsize'] = (15,9)

In [2]:
p1_coords = np.loadtxt('position1_coords.txt')
p2_coords = np.loadtxt('position2_coords.txt')
refstar1_coords = np.loadtxt('refstar1_coords.txt')
refstar2_coords = np.loadtxt('refstar2_coords.txt')

In [3]:
apsize1 = np.loadtxt('aperture_size_1.txt')
apsize2 = np.loadtxt('aperture_size_2.txt')
ref1_apsize = np.loadtxt('ref1_apsize.txt')
ref2_apsize = np.loadtxt('ref2_apsize.txt')

In [4]:
def magnitude(filename, asteroid_position, aperture_size):
    image = pf.getdata(filename)
    positions = asteroid_position
    aperture = CircularAperture(positions, r = aperture_size)
    annulus_aperture = CircularAnnulus(positions, r_in = 20, r_out = 30)
    apers = [aperture, annulus_aperture]
    phot_table = aperture_photometry(image, apers)
    bkg_mean = phot_table['aperture_sum_1'] / annulus_aperture.area
    bkg_total = bkg_mean * aperture.area
    net_star = phot_table['aperture_sum_0'] - bkg_total
    
    snr = net_star/np.sqrt(net_star + bkg_total)
    
    mag_err = 2.5*np.log10(1+(1/snr))

    magnitudes = -2.5 * np.log10(net_star)
    
    return np.array([magnitudes[0], mag_err[0]])

In [5]:
mags1 = np.array([magnitude('red_asteroid/reduced_ucd_0'+str(i)+'.fits', p1_coords[i-222], apsize1[i-222]) for i in range(222,285)])
mags2 = np.array([magnitude('red_asteroid/reduced_ucd_0'+str(i)+'.fits', p2_coords[i-285], apsize2[i-285]) for i in range(285,336)])
refmags = np.array([magnitude('red_asteroid/reduced_ucd_0'+str(i)+'.fits', refstar1_coords[i-222], ref1_apsize[i-222]) for i in range(222,285)])
refmags2 = np.array([magnitude('red_asteroid/reduced_ucd_0'+str(i)+'.fits', refstar2_coords[i-285], ref2_apsize[i-285]) for i in range(285,336)])

In [6]:
allmags = np.concatenate((mags1, mags2))
allrefs = np.concatenate((refmags, refmags2))

In [105]:
np.savetxt('magnitudes.txt', allmags)

### finding 'apparent magnitude' to determine colour

In [7]:
allrefs[0,0]

-11.348784193762922

In [9]:
refVmag = 14.976
refVerr = 0.023
refBmag = 16.098 
refBerr = 0.074
r = 14.621
rerr = 0.042
I = 14.307
Ierr = 0.056

In [10]:
def ri2R(r, i, r_err, i_err):
    R = r - 0.2936 * (r - i) - 0.1439
    R_err = (R * (np.sqrt((r_err/r)**2 + (i_err/i)**2))) + 0.0072
    return np.array([R, abs(R_err)])

In [11]:
refRmag = ri2R(r,I,rerr,Ierr)
refRmag

array([14.3849096 ,  0.07704082])

In [12]:
zpV1 = [refVmag - allrefs[0,0]]
zpV2 = [refVmag - allrefs[i,0] for i in range(225-222,285-222)]
zpV3 = [refVmag - allrefs[i,0] for i in range(287-222, 336-222)]
allZPV = np.concatenate((zpV1, zpV2, zpV3))
finalZPV = np.mean(allZPV)

ZPV_err = np.std(allZPV)

In [13]:
ZPR = [refRmag[0] - allrefs[i,0] for i in (224-222,286-222)]
finalZPR = np.mean(ZPR)
finalZPR

ZPR_err = np.std(ZPR)
ZPR_err

0.046924452324514476

In [14]:
ZPB = [refBmag - allrefs[i,0] for i in (223-222,285-222)]
finalZPB = np.mean(ZPB)
finalZPB

ZPB_err = np.std(ZPB)
ZPB_err

0.0483587376911494

In [15]:
V1 = np.array([allmags[0,0] + finalZPV, np.abs(allmags[0,1])+np.abs(ZPV_err)])
V2 = np.array([allmags[284-222,0] + finalZPV, np.abs(allmags[284-222,1])+np.abs(ZPV_err)])

B1 = np.array([allmags[1,0] + finalZPB, np.abs(allmags[1,1])+np.abs(ZPB_err)])
B2 = np.array([allmags[285-222,0] + finalZPB, np.abs(allmags[285-222,1])+np.abs(ZPB_err)])

R1 = np.array([allmags[2,0] + finalZPR, np.abs(allmags[2,1])+np.abs(ZPR_err)])
R2 = np.array([allmags[286-222,0] + finalZPR, np.abs(allmags[286-222,1])+np.abs(ZPR_err)])

In [27]:
B2

array([15.84437188,  0.05680174])

In [17]:
VB1 = np.array([V1[0] - B1[0], np.abs(V1[1]) + np.abs(B1[1])])
VB1

array([-1.05534538,  0.21743943])

In [18]:
VB2 = np.array([V2[0] - B2[0], np.abs(V2[1]) + np.abs(B2[1])])
VB2

array([-0.96728963,  0.2176565 ])

In [19]:
VR1 = np.array([V1[0] - R1[0], np.abs(V1[1]) + np.abs(R1[1])])
VR1

array([0.22034807, 0.21196538])

In [20]:
VR2 = np.array([V2[0] - R2[0], np.abs(V2[1]) + np.abs(R2[1])])
VR2

array([0.4593627 , 0.21202067])

In [21]:
BR1 = np.array([B1[0] - R1[0], np.abs(B1[1]) + np.abs(R1[1])])
BR1

array([1.27569345, 0.10817435])

In [22]:
BR2 = np.array([B2[0] - R2[0], np.abs(B2[1]) + np.abs(R2[1])])
BR2

array([1.42665233, 0.10796765])

### Differential photometry for lightcurve

In [250]:
def DiffPhot(ast_mag, ast_err, ref_mag, ref_err):
    DF = ref_mag - ast_mag
    DFerr = np.abs(ref_err) + np.abs(ast_err)
    return [DF, DFerr]

In [251]:
diff = np.array([DiffPhot(allmags[i][0], allmags[i][1], allrefs[i][0], allrefs[i][1]) for i in range(len(allmags))])

In [252]:
np.savetxt('diff_magnitudes.txt', diff)